In [33]:
import h5py
from dataset.dataset import Dataset
import argparse
import json
from preprocess.preprocess import Preprocess
parser = argparse.ArgumentParser(description='Train Example')
parser.add_argument('--config_path', type=str,
                    default='./config/competition.json')
args = parser.parse_args()

with open(args.config_path, 'r') as file:
    p_args = argparse.Namespace()
    p_args.__dict__.update(json.load(file))
    args = parser.parse_args(namespace=p_args)

pre_dataset = Preprocess(args = args)
bat_dict1, bat_dict2, bat_dict3 = pre_dataset.data_preprocess()
# calling function to load from disk
all_batches_dict = Dataset.load_batches_to_dict(bat_dict1, bat_dict2, bat_dict3)
# function to build features for ML
features_df = Dataset.build_feature_df(all_batches_dict)

<ipython-input-33-8fca77755924>:3: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File('./repo_template/dataset/2017-06-30_batchdata_updated_struct_errorcorrect.mat')


In [34]:
import numpy as np
batch = f['batch']
num_cells = batch['summary'].shape[0]
bat_dict = {}
for i in range(num_cells):
    cl = f[batch['cycle_life'][i, 0]].value
    policy = f[batch['policy_readable'][i, 0]].value.tobytes()[::2].decode()
    summary_IR = np.hstack(f[batch['summary'][i, 0]]['IR'][0, :].tolist())
    summary_QC = np.hstack(f[batch['summary'][i, 0]]['QCharge'][0, :].tolist())
    summary_QD = np.hstack(f[batch['summary'][i, 0]]['QDischarge'][0, :].tolist())
    summary_TA = np.hstack(f[batch['summary'][i, 0]]['Tavg'][0, :].tolist())
    summary_TM = np.hstack(f[batch['summary'][i, 0]]['Tmin'][0, :].tolist())
    summary_TX = np.hstack(f[batch['summary'][i, 0]]['Tmax'][0, :].tolist())
    summary_CT = np.hstack(f[batch['summary'][i, 0]]['chargetime'][0, :].tolist())
    summary_CY = np.hstack(f[batch['summary'][i, 0]]['cycle'][0, :].tolist())
    summary = {'IR': summary_IR, 'QC': summary_QC, 'QD': summary_QD, 'Tavg':
        summary_TA, 'Tmin': summary_TM, 'Tmax': summary_TX, 'chargetime': summary_CT,
                'cycle': summary_CY}
    cycles = f[batch['cycles'][i, 0]]
    cycle_dict = {}
    for j in range(cycles['I'].shape[0]):
        I = np.hstack((f[cycles['I'][j, 0]].value))
        Qc = np.hstack((f[cycles['Qc'][j, 0]].value))
        Qd = np.hstack((f[cycles['Qd'][j, 0]].value))
        Qdlin = np.hstack((f[cycles['Qdlin'][j, 0]].value))
        T = np.hstack((f[cycles['T'][j, 0]].value))
        Tdlin = np.hstack((f[cycles['Tdlin'][j, 0]].value))
        V = np.hstack((f[cycles['V'][j, 0]].value))
        dQdV = np.hstack((f[cycles['discharge_dQdV'][j, 0]].value))
        t = np.hstack((f[cycles['t'][j, 0]].value))
        cd = {'I': I, 'Qc': Qc, 'Qd': Qd, 'Qdlin': Qdlin, 'T': T, 'Tdlin': Tdlin, 'V': V, 'dQdV': dQdV, 't': t}
        cycle_dict[str(j)] = cd
        
    cell_dict = {'cycle_life': cl, 'charge_policy': policy, 'summary': summary, 'cycles': cycle_dict}
    key = 'b1c' + str(i)
    bat_dict[key] = cell_dict
    break
bat_dict1 = bat_dict

<ipython-input-34-5f8c7f647c6a>:6: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  cl = f[batch['cycle_life'][i, 0]].value
<ipython-input-34-5f8c7f647c6a>:7: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  policy = f[batch['policy_readable'][i, 0]].value.tobytes()[::2].decode()
<ipython-input-34-5f8c7f647c6a>:22: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  I = np.hstack((f[cycles['I'][j, 0]].value))
<ipython-input-34-5f8c7f647c6a>:23: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  Qc = np.hstack((f[cycles['Qc'][j, 0]].value))
<ipython-input-34-5f8c7f647c6a>:24: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  Qd = np.hstack((f[cycles['Qd'][j, 0]].value))
<ipython-input-34-5f8c7f647c6a>:25: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  Qdlin = np.hstack((f[

In [36]:
len(summary_IR)

326

In [27]:
key

'b1c0'